In [3]:
# toolbox
# tool box and maze_plotter
import numpy as np



##########-action constants-###################
N = 0
S = 1
E = 2
W = 3
NOOP = 4  


def discreteProb(p):
        # Draw a random number using probability table p (column vector)
        # Suppose probabilities p=[p(1) ... p(n)] for the values [1:n] are given, sum(p)=1 
        # and the components p(j) are nonnegative. 
        # To generate a random sample of size m from this distribution,
        #imagine that the interval (0,1) is divided into intervals with the lengths p(1),...,p(n). 
        # Generate a uniform number rand, if this number falls in the jth interval given the discrete distribution,
        # return the value j. Repeat m times.
        r = np.random.random()
        cumprob=np.hstack((np.zeros(1),p.cumsum()))
        sample = -1
        for j in range(p.size):
            if (r>cumprob[j]) & (r<=cumprob[j+1]):
                sample = j
                break
        return sample

def softmax(Q,x,tau):
    # Returns a soft-max probability distribution over actions
    # Inputs :
    # - Q : a Q-function represented as a nX times nU matrix
    # - x : the state for which we want the soft-max distribution
    # - tau : temperature parameter of the soft-max distribution
    # Output :
    # - p : probability of each action according to the soft-max distribution
    
    p = np.zeros((len(Q[x])))
    sump = 0
    for i in range(len(p)) :
        p[i] = np.exp((Q[x,i]/tau).round(5))
        sump += p[i]
    
    p = p/sump
    
    return p



def compare(V,Q,pol): 
    # compares the state value V with the state-action value Q following policy pol
    epsilon = 0.01 # precision of the comparison
    sumval = np.zeros((V.size))
    for i in range(V.size): # compute the difference between V and Q for each state
        sumval[i] = abs(V[i] - Q[i,pol[i]])
         
    if np.max(sumval)<epsilon :
        return True
    else :
        return False




# Reinforcement learning functions

By contrast with dynamic programming functions, the reinforcement learning functions are applied when the MDP is unknown.
More precisely, the state and action spaces are known, but the agent does not know the transition nor the reward function.

In this notebook we focus on *model free* reinforcement learning, the model-based case is treated [here](mbrl.ipynb).

## TD learning

Given a state and an action spaces as well as a policy, TD(0) computes the state value of this policy based on the following equation: 
    $$V^{(t+1)}(x_t) = V^{(t)}(x_t) + \alpha\delta_t,$$
    
where $\delta_t = r(x_t,u_t) + \gamma V^{(t)}(x_{t+1})-V^{(t)}(x_t)$ is the TD error and $\alpha$ is a parameter called "learning rate".</span>


The code is provided below, so that you can take inspiration later on. The important part is the computation of $\delta$, and the update of the values of $V$.

Once you have understood the code, write in the cell just below the code to run it.

Hint: to run TD learning, you need a policy as input. You can retreive such a policy by using the policy iteration method defined in the [dynamic_programming.ipynb](dynamic_programming.ipynb) notebook. Just import it, call it, and use the resulting policy as evaluated policy.


In [1]:
import numpy as np

# Given state and action spaces and a policy, it computes the state value of this policy

def TD(mdp, pol, nEpisodes=100000, nTimesteps=25, render=True):
    V = np.zeros((mdp.observation_space.size)) #initial state value V
    V_list = [V.copy()]
    alpha = 0.1 #learning rate
    mdp.timeout = 25 #sets timeout of an episode (maximum number of timesteps) - default set to 50
    
    if render:
        mdp.new_render()
    
    for i in range(nEpisodes) : #for each episode
        
        # Draw an initial state randomly (if uniform is set to False, the state is drawn according to the P0 
        #                                 distribution)
        x = mdp.reset(uniform=True) 
        done = mdp.done()
        while not done: #update episode at each timestep
            # Show agent
            if render:
                mdp.render(V, pol)
            
            # Step forward following the MDP: x=current state, 
            #                                 pol[i]=agent's action according to policy pol, 
            #                                 r=reward gained after taking action pol[i], 
            #                                 done=tells whether the episode ended, 
            #                                 and info gives some info about the process
            [y,r,done,info] = mdp.step(pol[x]) 
            
            # Update the state value of x
            if x in mdp.terminal_states:
                V[x] = r
            else:
                delta = r+mdp.gamma*V[y]-V[x]
                V[x] = V[x]+alpha*delta
            
            # Update agent's position (state)
            x=y
        
        # After each episode, we save the computed state value V
        V_list.append(V.copy()) 
    
    if render :
        mdp.render(V, pol)
    
    return V_list

In [ ]:
"\\write here the code to call the above TD function"

In [2]:
# visualize your final result
%matplotlib notebook

m.new_render()
m.render(V_list[-1], polPI_list[-1]) # V_list is the list of the V values of each episode, and same goes with polPI_list (policies)

NameError: name 'm' is not defined

## Q-learning

The QLearning algorithm accounts for an agent exploring an MDP and updating at each step a model of the state action-value function stored into a Q-table. It is updated as follows:
    $$Q^{(t+1)}(x_t,u_t) = Q_{(t)}(x_t,u_t) + \alpha \delta_t,$$
    
and the temporal difference error is processed using $\delta_t = r(x_t,u_t) + \gamma \max_{u_{t+1} \in A} Q^{(t)}(x_{t+1},u_{t+1})-Q^{(t)}(x_t,u_{t})$

The cell below gives the code of Q-learning, where you must just write the central update rule.


In [6]:
import numpy as np

########################### Q-Learning ###########################
    
# Given a temperature "tau", the QLearning function computes the state action-value function 
# based on a softmax policy 
def QLearning(mdp,tau,nEpisodes=100000,nTimesteps=50,alpha=0.01,render=True):
    # Initialize the state-action value function
    # alpha is the learning rate
    Q = np.zeros((mdp.observation_space.size,mdp.action_space.size))
    
    Q_list = []
    policy_list = []
    
    # Run learning cycle
    mdp.timeout = nTimesteps #episode length
    
    if render:
        mdp.new_render()
        
    for i in range(nEpisodes) :
        #Draw the first state of episode i using a uniform distribution over all the states
        x = mdp.reset(uniform=True) 
        done = mdp.done()
        while not done:
            if render :
                # Show the agent in the maze
                mdp.render(Q, Q.argmax(axis=1))
            
            # Draw an action using a soft-max policy
            u = mdp.action_space.sample(prob_list=softmax(Q,x,tau))

            # Perform a step of the MDP
            [y,r,done,info] = mdp.step(u)

            # Update the state-action value function with Q-Learning
            if x in mdp.terminal_states:
                Q[x,u] = "\\..."
            else:
                delta = "\\..."
                Q[x,u] = Q[x,u] + "\\..."
            
            # Update agent's position
            x = y
            
        # Save state-action value after each episode
        Q_list.append(Q.copy())
        policy_list.append(Q.argmax(axis=1))

    if render :
        # Show the agent in the maze
        mdp.render(Q, Q.argmax(axis=1))
    return [Q_list, policy_list]

Now, write the code to run Q-learning below

In [ ]:
"run the Q-learning code here"

In [7]:
# Visualize your results

# In DP, the rendering was done in real time, meaning that as the Q values were being processed, the maze environment
# was updated, even for rather large mazes. For RL algorithms, convergence is slower, 
# so we suggest you to build a much smaller maze, using for instance:
# walls = [6, 13, 14, 15]
# height = 4
# width = 5
# terminal_states=[width*height-1]
# m = maze_mdp(width, height, walls=walls, terminal_states=terminal_states)

# As output, lists of the Q values and policies are stored and can be used either to show the result offline
# or to generate videos that you can show on the notebook or save into your disk

%matplotlib notebook

from IPython.display import HTML  # used to display the video on the notebook

m.new_render()
m.render(QPI_list[-1], polPI_list[-1])  # QPI_list is the list of the Q values processed at each episode, 
# polPi_list is the list of the policies

# you can generate a video of your learning process using those lists
# nb_frames = 100 # number of frames of your video
# ani = m.create_animation(QPI_list, polPI_list, nb_frames) # generate your video
# HTML(ani.to_jshtml()) # show your video on the notebook (with a video widget than enables you to manage your output)
# ani.save("Q_learning.mp4") # save your video

NameError: name 'm' is not defined

 ### Learning dynamics
    
If you watch carefully the values while the agent is learning, you will see that the agent favors certains paths over others which have a strictly equivalent value. This can be explained easily: as the agent chooses a path for the first time, it updates the values along that path, these values get higher than the surrounding values, and the agent will choose the same path again and again, increasing the phenomenon. Only steps of random exploration can counterbalance this effect, but they do so extremely slowly.

### Effect of hyper-parameters

There are three hyper-parameters in Q-learning: the softmax temperature $\tau$, the learning rate $\alpha$, and the discount factor $\gamma$. Using a small maze, try various values for these hyper-parameters and explain what is happenning.



### Exploration


In the code above, action selection is based on a soft-max policy. Instead, it could have relied on *epsilon-greedy*.
Copy-paste the above code below and do the replacement.

In [ ]:
"copy-paste and replace here"

## SARSA

The SARSA algorithm is very similar to Q_learning. At first glance, the only difference is in the update rule. However, to perform the update in SARSA, one needs to know the action the agent will take when it will be at the next state, even if the agent is taking a random action.

This implies that the next state action is determined in advance and stored for being played at the next time step.


By taking inspiration from the above Qlearning function, write in the cell below a Sarsa function that implements the corresponding algorithm. Then write the code to run it in the cell after.

In [ ]:
"write SARSA here"

In [ ]:
"run SARSA here"

In [ ]:
# visualize your results

%matplotlib notebook

m.new_render()
m.render(QS_list[-1], polS_list[-1]) # output the last Q values and policy

## Wrapping up

Compare the number of steps needed by Qlearning and Sarsa to converge on the given MDP. To figure out, add a counter of number of steps to your various algorithms, and run them for a given number of steps (for instance, 10.000). Then watch the corresponding Q-table: can you determine if one was updated more than another? Eventually, do so with much smaller mazes...